# Pytorch basics

Adapted from [Pytorch tutorial for Deep Learning researchers](https://github.com/yunjey/pytorch-tutorial) (Yunvey Choi, 2018).

Used as part of Deep Learning, Gilles Louppe, 2018.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch 
import torchvision
import torch.nn as nn
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Basic autograd example 1

In [ ]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [ ]:
# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

In [ ]:
y1 = torch.tensor(1., requires_grad=True)

In [ ]:
print(y1.grad)

<div class="alert alert-success">
<b>EXERCISE</b>:

<ul>
    <li>Define a polynomial model <code>y = w1*x + w2*x^2 + b</code>.
    <li>Compute the value of <code>y</code> at <code>x=2</code>, <code>w1=1.5</code> and <code>w2=-1.0</code>.
    <li>Evaluate the derivate of <code>y</code> with respect to <code>w2</code> at these values.
</ul>

</div>

# Basic autograd example 2

In [ ]:
# Create tensors.
x = torch.randn(5, 3)
y = torch.randn(5, 2)

# Build a linear layer.
linear = nn.Linear(3, 2)
print('w: ', linear.weight)
print('b: ', linear.bias)

In [ ]:
for p in linear.parameters():
    print(p, p.numel())

In [ ]:
# Forward propagation.
pred = linear(x)
print(pred)

In [ ]:
# Build Loss and Optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss)

In [ ]:
# Backpropagation.
loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

In [ ]:
# 1-step Optimization (gradient descent).
optimizer.step()

# You can also do optimization at the low level as shown below.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after optimization.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

<div class="alert alert-success">
<b>EXERCISE</b>:

Write the code above within a for loop that trains the linear models for 100 steps.

</div>

# Load data from numpy

In [ ]:
a = np.array([[1,2], [3,4]])
b = torch.from_numpy(a)      # convert numpy array to torch tensor
c = b.numpy()                # convert torch tensor to numpy array

# Implementing the input pipeline

In [ ]:
# Download and construct dataset.
train_dataset = dsets.CIFAR10(root='./data/',
                              train=True, 
                              transform=transforms.ToTensor(),
                              download=True)

In [ ]:
# Select one data pair (read data from disk).
image, label = train_dataset[7]
print(image.size())
print(label)

In [ ]:
from scipy.misc import toimage
toimage(image)

In [ ]:
# Data Loader (this provides queue and thread in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True,
                                           num_workers=2)

# When iteration starts, queue and thread start to load dataset from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of data loader is as below.
for images, labels in train_loader:
    # Your training code will be written here
    pass

# Input pipeline for custom dataset

In [ ]:
# You should build custom dataset as below.
class CustomDataset(data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file path or list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# Then, you can just use prebuilt torch's data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=100, 
                                           shuffle=True,
                                           num_workers=2)

# Save and load model

In [ ]:
# Save and load the entire model.
torch.save(resnet, 'model.pkl')
model = torch.load('model.pkl')

# Save and load only the model parameters(recommended).
torch.save(resnet.state_dict(), 'params.pkl')
resnet.load_state_dict(torch.load('params.pkl'))